In [1]:
import time
import pandas as pd
import numpy as np
import math
from statistics import mean
import statistics
import itertools
from scipy.stats import norm


In [2]:
# item: 8010903
daily_demands = {'1': [(12.6,2.6), (13.9,4.0), (11.5,4.1), (10.7,4.3), (10.3,4.4), (9.1,4.4), (12.3,4.5)],
                '2': [(1.5,0.6), (1.5,0.9), (1.5,0.9), (1.5,1.0), (10.3,4.4), (9.1,4.4), (12.3,4.5)],
                '3': [(3.5,1.3), (3.3,2.0), (3.6,2.0), (3.1,2.1), (3.7,2.1), (3.7,2.0), (3.9,2.0)],
                '4': [(2.7,2.6), (12.8,3.9), (15.0,4.0), (14.5,4.3), (18.8,4.3), (17.8,4.3), (26.2,4.3)], # 3889
                '5': [(6.6,2.1), (6.4,3.2), (5.9,3.1), (5.0,3.3), (6.3,3.3), (6.1,3.2), (6.7,3.2)], # 3349
                '6': [(4.0,1.8), (4.8,2.7), (7.5,2.7), (7.4,2.7), (6.6,2.8), (5.6,2.8), (9.4,2.8)], # 3328
                '7': [(27.6,3.4), (32.8,5.0), (19.3,5.3), (15.1,5.8), (18.9,5.8), (17.3,6.1), (30.6,5.8)], # 2189
                '8': [(17.3,3.3), (11.0,5.1), (10.2,5.1), (9.4,5.3), (11.2,5.2), (11.0,5.3), (13.2,5.2)], # 2177
                '9': [(66.6,5.8), (55.4,8.4), (42.6,8.8), (35.7,10.0), (45.5,10.0), (42.0,10.2), (43.6,10.1)]} # 1096 

In [3]:
current_inventory = {'1': 81,
                    '2': 38,
                    '3': 25,
                    '4': 200,
                    '5': 43,
                    '6': 90,
                    '7': 162,
                    '8': 20,
                    '9': 200}



In [4]:
initial_inventory = current_inventory.copy()

In [5]:
def reset_inventory():
    return initial_inventory.copy()

In [6]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [7]:
num_stores = 9
current_day = 0

In [8]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [9]:
convolution('1', current_day)

(80.4, 10.818040487999664)

In [10]:
p_value = (costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio=norm.ppf(p_value)


In [11]:
current_day = 0
sales_dff = pd.DataFrame(columns=['Store', 'Day', 'Sales','Lost Sales','Demand','Inventory'])

In [12]:
def update_inventory(i,current_day):
    global sales_dff
    todays_sales = {i: max(0,int(np.random.normal(daily_demands[(i)][current_day][0], daily_demands[(i)][current_day][1])))
                    }
    beforesales_inv=current_inventory[i]
    actual_sales = min(todays_sales[i], current_inventory[i])
    lost_sales = max(0, todays_sales[i] - current_inventory[i])
    current_inventory[i] = current_inventory[i]-actual_sales
    new_row = pd.DataFrame({'Store': [i],"Week":[week], 'Day': [current_day], 'Sales': [actual_sales],'Lost Sales':lost_sales,'Demand':todays_sales[i],'Inventory':current_inventory[i],'Inv bef sales': beforesales_inv})
   
    sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)
    
    return current_inventory   

In [13]:
def reset_inventory():
    return initial_inventory.copy()

In [14]:
def shipper_candidates(current_day):
    shipper_cand = []
   
    
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean + (ratio*st_dev)
        
        if critical_point < update_inventory(str(i),current_day)[i]:
            shipper_cand.append(i)
    return shipper_cand

In [15]:
def receiving_candidates(current_day):
    receiving_cand = []
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean - (ratio*st_dev)
      
        if critical_point > current_inventory[i]:
            receiving_cand.append(i)
    return receiving_cand


In [16]:
def converge_parwise(cost_list, threshold):
    pairwise_delta = [abs(a - b) for a, b in itertools.combinations(cost_list, 2)]
    max_delta = max(pairwise_delta)
    
    return max_delta < threshold


In [17]:
current_time=time.time()
max_week=2000
week = 1
sim_check=100
threshold=0.5
results = pd.DataFrame()
resultsale = pd.DataFrame()
cost_week=[]
expected_costs = []
while (week<=max_week):
    for _ in range(sim_check):
    
        print(f"hafta ------------------- {week}")
        current_inventory = reset_inventory()
        #print(f"{week}. haftanın başlangıcı, inventory: {current_inventory}")
        weekly_df = pd.DataFrame(columns=["week", "day", "shipper", "receiver", "transshipment_amount"])
        sales_dff = pd.DataFrame()
        current_day = 0
        for i in range(6):
            if current_day <= 5:

                empty_list=[]

                shipping_list = shipper_candidates(current_day)
                receiving_list = receiving_candidates(current_day)

                daily_df = pd.DataFrame(columns=["shipper", "receiver", "transshipment_amount", "transshipment_cost"])

                for i in shipping_list:
                    mean,stdev=convolution(i,current_day)            

                    for k in receiving_list:
                        mean2,stdev2=convolution(k,current_day)
                        best_x = 0
                        min_cost = float('inf')
                        s_inv=current_inventory[i]
                        r_inv=current_inventory[k]

                        #print("shipper inventory",s_inv,"mağaza no:",i)
                        #print("receiver inventory",r_inv,"mağaza no",k)


                        x_lower=0
                        x_upper=s_inv
                        tolerance=1
                        epsilon=1
                        min_cost=float("inf")
                        best_x=None

                        while (x_upper-x_lower>tolerance):
                            mid=(x_upper+x_lower)//2
                            mid_left=max(mid-epsilon,x_lower)
                            cost_eps =0
                            mid_eps = min(mid + epsilon, x_upper)


                            sum_cost=0
                            new_list=[]
                            hold_cv={}


                            for number in range(100):
                                (exp_shipping)=int(np.random.normal(mean,stdev))
                                (exp_receiver)=int(np.random.normal(mean2,stdev2))

                                cost = (costs['stockout'] * max(0, exp_receiver - (current_inventory[k] + mid_left))) + \
                               (costs['stockout'] * max(0, exp_shipping - (current_inventory[i] - mid_left))) + \
                               (costs['transshipment'] * mid_left)
                                sum_cost+=cost
                                new_list.append(cost)





                            expected_cost=sum_cost/100
                            st_dev=np.std(new_list)

                            CV=(st_dev/expected_cost)*100
                            hold_cv[mid]=CV








                            cost_eps =0
                            mid_eps = min(mid + epsilon, x_upper)
                            for number in range(100):
                                exp_shipping=np.random.normal(mean,stdev)
                                exp_receiver=np.random.normal(mean2,stdev2)

                                cost2 = (costs['stockout'] * max(0, exp_receiver - (current_inventory[k] + mid_eps))) + \
                               (costs['stockout'] * max(0, exp_shipping - (current_inventory[i] - mid_eps))) + \
                               (costs['transshipment'] * mid_eps)
                                cost_eps+=cost2
                            expected_costeps=cost_eps/100
                            if (expected_cost<expected_costeps):
                                x_upper=mid
                                if (expected_cost<min_cost):
                                    min_cost=expected_cost
                                    best_x=mid_left
                            else:
                                x_lower=mid
                                if (expected_costeps<min_cost):
                                    min_cost=expected_costeps
                                    best_x=mid_eps









                        daily_df = pd.concat([daily_df, pd.DataFrame([{
                        "shipper": i,
                        "receiver": k,
                        "transshipment_amount": best_x,
                        "transshipment_cost": min_cost
                }])], ignore_index=True)


                while not daily_df.empty:

                    min_idx = daily_df["transshipment_cost"].idxmin()
                    min_cost_row = daily_df.loc[min_idx]
                    deleted_shipper = min_cost_row["shipper"]
                    deleted_receiver = min_cost_row["receiver"]
                    transhipment_amount = daily_df.loc[min_idx, "transshipment_amount"]

                    current_inventory[str(deleted_shipper)] = current_inventory[str(deleted_shipper)] - int(transhipment_amount)
                    current_inventory[str(deleted_receiver)] = current_inventory[str(deleted_receiver)] + int(transhipment_amount)

                    weekly_df = pd.concat([weekly_df, pd.DataFrame([{
                        "week": week,
                        "day": current_day,
                        "shipper": deleted_shipper,
                        "receiver": deleted_receiver,
                        "transshipment_amount": transhipment_amount

                }])], ignore_index=True)

                    daily_df = daily_df[(daily_df["shipper"] != deleted_shipper) & (daily_df["receiver"] != deleted_receiver)]

                #print(f"biten gün: {current_day}, inventory_levels: {current_inventory}") 
                #print("-------------------------------")

                current_day = current_day + 1


        if current_day == 6:
            for i in current_inventory.keys():

                mean,stdev=daily_demands[i][current_day]
                number=int(np.random.normal(mean,stdev))
                beforesales_inv=current_inventory[i]
                actual_sales = min(number, current_inventory[i])
                lost_sales = max(0, number - current_inventory[i])
                current_inventory[i]=current_inventory[i]-actual_sales
                new_row = pd.DataFrame({'Store': [i],"Week":[week], 'Day': [current_day], 'Sales': actual_sales,'Lost Sales':lost_sales,'Demand':number,'Inventory':current_inventory[i],'Inv bef sales': beforesales_inv})
                sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)
        #print(sales_dff)
        weekly_df["transshipment_amount"] = pd.to_numeric(weekly_df["transshipment_amount"], errors="coerce")
        sales_dff["Inventory"] = pd.to_numeric(sales_dff["Inventory"], errors="coerce")
        sales_dff["Lost Sales"]=pd.to_numeric(sales_dff["Lost Sales"], errors="coerce")
        weekly_cost = int(costs["transshipment"])*weekly_df["transshipment_amount"].sum()+int(costs["stockout"])*sales_dff["Lost Sales"].sum()+int(costs["holding"])*sales_dff["Inventory"].sum()
        weekly_df["week_cost"] = weekly_cost
        #print(weekly_df)
        results = pd.concat([results, weekly_df], ignore_index=True)
        resultsale=pd.concat([resultsale, sales_dff], ignore_index=True)
        cost_week.append(weekly_cost)
        week = week + 1
        
        
    current_expected = np.mean(cost_week)
    expected_costs.append(current_expected)
    print(f"{week-1} iterasyonda expected cost: {current_expected:.2f}")
   
    if len(expected_costs) >= 2:
        delta = abs(expected_costs[-1] - expected_costs[-2])
        print("Delta:",delta)
        if delta<=threshold:
            print(f" Convergence sağlandı. {week} iterasyonda durdu.")
            break

print(resultsale)
print(expected_costs)            
print("Passing time to run the program: ",time.time()-current_time)
    

hafta ------------------- 1
hafta ------------------- 2
hafta ------------------- 3
hafta ------------------- 4
hafta ------------------- 5
hafta ------------------- 6
hafta ------------------- 7
hafta ------------------- 8
hafta ------------------- 9
hafta ------------------- 10
hafta ------------------- 11
hafta ------------------- 12


/var/folders/1d/t639nh391qzc2mr0903r897r0000gn/T/ipykernel_1844/649883069.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  daily_df = pd.concat([daily_df, pd.DataFrame([{
/var/folders/1d/t639nh391qzc2mr0903r897r0000gn/T/ipykernel_1844/649883069.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  daily_df = pd.concat([daily_df, pd.DataFrame([{
/var/folders/1d/t639nh391qzc2mr0903r897r0000gn/T/ipykernel_1844/649883069.py:119: FutureWarning: The behavior of DataFrame concatenation with empty 

hafta ------------------- 13
hafta ------------------- 14
hafta ------------------- 15
hafta ------------------- 16
hafta ------------------- 17
hafta ------------------- 18
hafta ------------------- 19
hafta ------------------- 20
hafta ------------------- 21
hafta ------------------- 22
hafta ------------------- 23
hafta ------------------- 24
hafta ------------------- 25
hafta ------------------- 26


/var/folders/1d/t639nh391qzc2mr0903r897r0000gn/T/ipykernel_1844/649883069.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  daily_df = pd.concat([daily_df, pd.DataFrame([{
/var/folders/1d/t639nh391qzc2mr0903r897r0000gn/T/ipykernel_1844/649883069.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  daily_df = pd.concat([daily_df, pd.DataFrame([{
/var/folders/1d/t639nh391qzc2mr0903r897r0000gn/T/ipykernel_1844/649883069.py:119: FutureWarning: The behavior of DataFrame concatenation with empty 

hafta ------------------- 27
hafta ------------------- 28
hafta ------------------- 29
hafta ------------------- 30
hafta ------------------- 31
hafta ------------------- 32
hafta ------------------- 33
hafta ------------------- 34
hafta ------------------- 35
hafta ------------------- 36
hafta ------------------- 37
hafta ------------------- 38
hafta ------------------- 39
hafta ------------------- 40
hafta ------------------- 41
hafta ------------------- 42
hafta ------------------- 43
hafta ------------------- 44


...

hafta ------------------- 1077
hafta ------------------- 1078
hafta ------------------- 1079
hafta ------------------- 1080
hafta ------------------- 1081
hafta ------------------- 1082
hafta ------------------- 1083
hafta ------------------- 1084
hafta ------------------- 1085
hafta ------------------- 1086
hafta ------------------- 1087
hafta ------------------- 1088
hafta ------------------- 1089
hafta ------------------- 1090
hafta ------------------- 1091
hafta ------------------- 1092
hafta ------------------- 1093
hafta ------------------- 1094
hafta ------------------- 1095
hafta ------------------- 1096
hafta ------------------- 1097
hafta ------------------- 1098
hafta ------------------- 1099
hafta ------------------- 1100
1100 iterasyonda expected cost: 5564.38
Delta: 9.099454545455046
hafta ------------------- 1101
hafta ------------------- 1102
hafta ------------------- 1103
hafta ------------------- 1104
hafta ------------------- 1105
hafta ------------------- 1106
hafta

hafta ------------------- 1337
hafta ------------------- 1338
hafta ------------------- 1339
hafta ------------------- 1340
hafta ------------------- 1341
hafta ------------------- 1342
hafta ------------------- 1343
hafta ------------------- 1344
hafta ------------------- 1345
hafta ------------------- 1346
hafta ------------------- 1347
hafta ------------------- 1348
hafta ------------------- 1349
hafta ------------------- 1350
hafta ------------------- 1351
hafta ------------------- 1352
hafta ------------------- 1353
hafta ------------------- 1354
hafta ------------------- 1355
hafta ------------------- 1356
hafta ------------------- 1357
hafta ------------------- 1358
hafta ------------------- 1359
hafta ------------------- 1360
hafta ------------------- 1361
hafta ------------------- 1362
hafta ------------------- 1363
hafta ------------------- 1364
hafta ------------------- 1365
hafta ------------------- 1366
hafta ------------------- 1367
hafta ------------------- 1368
hafta --

hafta ------------------- 1599
hafta ------------------- 1600
1600 iterasyonda expected cost: 5553.79
Delta: 10.589708333332965
hafta ------------------- 1601
hafta ------------------- 1602
hafta ------------------- 1603
hafta ------------------- 1604
hafta ------------------- 1605
hafta ------------------- 1606
hafta ------------------- 1607
hafta ------------------- 1608
hafta ------------------- 1609
hafta ------------------- 1610
hafta ------------------- 1611
hafta ------------------- 1612
hafta ------------------- 1613
hafta ------------------- 1614
hafta ------------------- 1615
hafta ------------------- 1616
hafta ------------------- 1617
hafta ------------------- 1618
hafta ------------------- 1619
hafta ------------------- 1620
hafta ------------------- 1621
hafta ------------------- 1622
hafta ------------------- 1623
hafta ------------------- 1624
hafta ------------------- 1625
hafta ------------------- 1626
hafta ------------------- 1627
hafta ------------------- 1628
haft

hafta ------------------- 1857
hafta ------------------- 1858
hafta ------------------- 1859
hafta ------------------- 1860
hafta ------------------- 1861
hafta ------------------- 1862
hafta ------------------- 1863
hafta ------------------- 1864
hafta ------------------- 1865
hafta ------------------- 1866
hafta ------------------- 1867
hafta ------------------- 1868
hafta ------------------- 1869
hafta ------------------- 1870
hafta ------------------- 1871
hafta ------------------- 1872
hafta ------------------- 1873
hafta ------------------- 1874
hafta ------------------- 1875
hafta ------------------- 1876
hafta ------------------- 1877
hafta ------------------- 1878
hafta ------------------- 1879
hafta ------------------- 1880
hafta ------------------- 1881
hafta ------------------- 1882
hafta ------------------- 1883
hafta ------------------- 1884
hafta ------------------- 1885
hafta ------------------- 1886
hafta ------------------- 1887
hafta ------------------- 1888
hafta --

hafta ------------------- 2116
hafta ------------------- 2117
hafta ------------------- 2118
hafta ------------------- 2119
hafta ------------------- 2120
hafta ------------------- 2121
hafta ------------------- 2122
hafta ------------------- 2123
hafta ------------------- 2124
hafta ------------------- 2125
hafta ------------------- 2126
hafta ------------------- 2127
hafta ------------------- 2128
hafta ------------------- 2129
hafta ------------------- 2130
hafta ------------------- 2131
hafta ------------------- 2132
hafta ------------------- 2133
hafta ------------------- 2134
hafta ------------------- 2135
hafta ------------------- 2136
hafta ------------------- 2137
hafta ------------------- 2138
hafta ------------------- 2139
hafta ------------------- 2140
hafta ------------------- 2141
hafta ------------------- 2142
hafta ------------------- 2143
hafta ------------------- 2144
hafta ------------------- 2145
hafta ------------------- 2146
hafta ------------------- 2147
hafta --

In [18]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8097 entries, 0 to 8096
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   week                  8097 non-null   object
 1   day                   8097 non-null   object
 2   shipper               8097 non-null   object
 3   receiver              8097 non-null   object
 4   transshipment_amount  8097 non-null   int64 
 5   week_cost             8097 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 379.7+ KB


In [19]:
results.head()

,week,day,shipper,receiver,transshipment_amount,week_cost
0,1,0,4,7,50,6150
1,1,0,6,8,62,6150
2,1,1,4,6,19,6150
3,1,2,7,9,114,6150
4,2,0,4,8,99,4661


In [60]:
results["transshipment_amount"].sum()/2000



181.945

In [58]:
resultsale["Inventory"].sum()/2000

2637.181

In [56]:
resultsale["Lost Sales"].sum()/2000

89.249

In [23]:
unique_week_costs = results.drop_duplicates(subset="week")[["week_cost"]]

mean_cost = unique_week_costs["week_cost"].mean()
deviation_cost = unique_week_costs["week_cost"].std()

CV=(deviation_cost/mean_cost)*100
print(CV)
print(mean_cost)

17.042517862374655
5496.596


In [24]:
# 100:   3623.39, cv: 10.39

# 500:   3664.55, cv: 13.06

# 1000:  3620.57, cv: 12.46

# 5000:  3630.35, cv: 12.33 , run time: 232.79 secs

# 7000:  3622.53, cv: 12.53, run time: 324 secs

# 10000: 3625.02, cv: 12.45, run time: 460.96 secs

# 15000: , cv: , run time: 

In [25]:
results["transshipment_amount"].sum()

363890

In [26]:
resultsale.head(63)

,Store,Week,Day,Sales,Lost Sales,Demand,Inventory,Inv bef sales
0,1,1,0,15,0,15,66,81
1,2,1,0,1,0,1,37,38
2,3,1,0,7,0,7,18,25
3,4,1,0,4,0,4,196,200
4,5,1,0,6,0,6,37,43
...,...,...,...,...,...,...,...,...
58,5,1,6,0,7,7,0,0
59,6,1,6,10,0,10,10,20
60,7,1,6,0,37,37,0,0
61,8,1,6,14,2,16,0,14
